# Introduction to Decision Trees

In [37]:
import pandas

# Set index_col to False to avoid pandas thinking that the first column is row indexes (it's age)
income = pandas.read_csv("data/income.csv", index_col=False, skipinitialspace=True)
income.head(5)

,age,workclass,flnwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,high-income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


* age: continuous.
* workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
* fnlwgt: continuous.
* education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
* education-num: continuous.
* marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
* occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
* relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
* race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
* sex: Female, Male.
* capital-gain: continuous.
* capital-loss: continuous.
* hours-per-week: continuous.
* native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
age, workclass, flnwgt, education, education-num, marital-status, occupation, relationship, race, sex, capital-gain, capital-loss, hours-per-week, native-country

Before we get started with decision trees, we need to convert the categorical variables in our data set to numeric variables. This involves assigning a number to each category label, then converting all of the labels in a column to the corresponding numbers.

In [38]:
# Convert categorical columns from text categories to numbers
for cat_col in ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'high-income']:
    col = pandas.Categorical(income[cat_col])
    income[cat_col] = col.codes

A decision tree is made up of a series of nodes and branches. A node is where we split the data based on a variable, and a branch is one side of the split.

Let's split income into two parts based on the value of the workclass column.

* private_incomes should contain all rows where workclass is 4.
* public_incomes should contain all rows where workclass is not 4.


In [39]:
private_incomes = income[income['workclass'] == 4]
public_incomes= income[income['workclass']!=4]

We're trying to predict the high_income column.
If high_income is 1, it means that the person has an income higher than 50k a year.
If high_income is 0, it means that they have an income less than or equal to 50k a year.

When we split, we'll try to separate as many 0s from 1s in the high_income column as we can. In order to do this, we need a metric for how "together" the different values in the high_income column are.

We can use a metric called **entropy** for this purpose. Entropy refers to disorder. The more "mixed together" 1s and 0s are, the higher the entropy. A data set consisting entirely of 1s in the high_income column would have low entropy.

$$-\sum_{i=1}^{c} {\mathrm{P}(x_i) \log_b \mathrm{P}(x_i)}$$

We iterate through each unique value in a single column (in this case, high_income), and assign it to . We then compute the probability of that value occurring in the data ($\mathrm{P}(x_i)$). Next we do some multiplication, and sum all of the values together.

is the base of the logarithm. We commonly use the value 2 for this, but we can also set it to 10 or another value.

Let's say we have this data:

|age  |  high_income|
|------|--------|
|25   |  1|
|50  |   1|
|30  |   0|
|50   |  0|
|80   |  1|

We could compute its entropy like this:

$$\begin{array}
-\sum_{i=1}^{c} {\mathrm{P}(x_i) \log_b \mathrm{P}(x_i)} & = & -((2/5 * \log_2 2/5) + (3/5 * \log_2 3/5)) \nonumber \\ 
& = & -(-0.5287712379549449 + -0.44217935649972373)\nonumber \\
& = & .97 \nonumber \\
\end{array}$$

In [40]:
import math
# We'll do the same calculation we did above, but in Python
# Passing in 2 as the second parameter to math.log will take a base 2 log
entropy = -(2/5 * math.log(2/5, 2) + 3/5 * math.log(3/5, 2))
print(entropy)

0.9709505944546686


Let's compute the entropy of the high_income column in the income dataframe, and assign the result to income_entropy

In [42]:
ones = len(income[income['high-income']==1])
total = income.shape[0]
zeros = total - ones
income_entropy = -(zeros/total * math.log(zeros/total, 2) + ones/total * math.log(ones/total, 2))

In [43]:
income_entropy

0.7963839552022132

To figure out which variable to split on, we can use information gain, which tells us which split will reduce entropy the most.

Here's the formula for information gain:

$$IG(T,A) = Entropy(T)-\sum_{v\in A}\frac{|T_{v}|}{|T|} \cdot Entropy(T_{v})$$

We're computing information gain (IG) for a given target variable (T), as well as a given variable we want to split on (A). 

To compute it, we first calculate the entropy for T. Then, for each unique value v in the variable A, we compute the number of rows in which A takes on the value v, and divide it by the total number of rows. Next, we multiply the results by the entropy of the rows where A is v. We add all of these subset entropies together, then subtract from the overall entropy to get information gain.

One strategy for constructing trees is to create as many branches at each node as there are unique values for the variable we're splitting on. So if the variable has three or four values, we'd end up with three or four branches. This approach usually involves more complexity than it's worth and doesn't improve prediction accuracy, but it's worth knowing about.

To simplify the calculation of information gain and make splits simpler, we won't do it for each unique value. We'll find the median for the variable we're splitting on instead. Any rows where the value of the variable is below the median will go to the left branch, and the rest of the rows will go to the right branch. To compute information gain, we'll only have to compute entropies for two subsets.


As an example:

|age  |  high_income|
|------|--------|
|25   |  1|
|50  |   1|
|30  |   0|
|50   |  0|
|80   |  1|

Let's say we wanted to split this data set based on age. First, we calculate the median age, which is 50. Then, we assign any row with a value less than or equal to the median age the value 0 (in a new column named split_age), and the other rows 1.

|age  |  high_income| split_age |
|------|--------|----------|
|25   |  1| 0 |
|50  |   1| 0|
|30  |   0| 0|
|50   |  0|0|
|80   |  1|1|

Now we compute entropy:
    
$$\begin{align}
IG(T,A) = Entropy(T)-\sum_{v\in A}\frac{|T_{v}|}{|T|} \cdot Entropy(T_{v}) \nonumber \\
\end{align}$$

$$ Entropy(T) = 0.97 $$
$$ T_{0} = 4 $$
$$ T_{1} = 1 $$
$$ T = 5 $$
$$ Entropy(T_{0}) = -((2/4 * \log_2 2/4) + (2/4 * \log_2 2/4))$$
$$ Entropy(T_{1}) = -((0/1 * \log_2 0/1) + (0/1 * \log_2 0/1))$$

Which gives:

$$\begin{align}
IG(T,A) = .97 - (((4/5) * -(1/2 * log_{2} 1/2 + 1/2 * log_{2} 1/2 )) + -(1/5 *  (0 * log_{2} 0 + 1 * log_{2} 1 ))) \nonumber \\
IG(T,A) = .97 - ((4/5) * -(-.5 + -.5)) + (1/5 *  -(0 + 1 * 0 )) \nonumber \\
IG(T,A) = .97 - (4/5) \nonumber \\
IG(T,A) = .169 \nonumber \\
\end{align}$$

Let's calculate the same using Python:

In [47]:
import numpy

def calc_entropy(column):
    """
    Calculate entropy given a pandas series, list, or numpy array.
    """
    # Compute the counts of each unique value in the column
    counts = numpy.bincount(column)
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    
    return -entropy

# Verify that our function matches our answer from earlier
entropy = calc_entropy([1,1,0,0,1])

information_gain = entropy - ((.8 * calc_entropy([1,1,0,0])) + (.2 * calc_entropy([1])))
print(information_gain)

0.170950594455


Let's compute the information gain for splitting on the age column of income.

* Let's first, compute the median of age.
* Then, assign anything less than or equal to the median to the left branch, and anything greater than the median to the right branch.
* Compute the information gain and assign it to age_information_gain.


In [57]:
import numpy as np
median_age = np.median(income['age'])
left_branch = income[income['age'] <= median_age]
right_branch = income[income['age'] > median_age]

total = income.shape[0]
age_zeros = len(left_branch)
age_ones = len(right_branch)
prop_zeros = age_zeros / total
prop_ones = age_ones / total

T_ageZeros = calc_entropy(left_branch['high-income'])
T_ageOnes = calc_entropy(right_branch['high-income'])

age_information_gain = income_entropy - (prop_zeros * T_ageZeros + prop_ones * T_ageOnes)

In [58]:
age_information_gain

0.047028661304691965

We can determine the best variable to split a node on. When we start our tree, we want to make an initial split. We'll find the variable to split on by calculating which split would have the highest information gain.

In [64]:
def calc_information_gain(data, split_name, target_name):
    """
    Calculate information gain given a data set, column to split on, and target
    """
    # Calculate the original entropy
    original_entropy = calc_entropy(data[target_name])
    
    # Find the median of the column we're splitting
    column = data[split_name]
    median = column.median()
    
    # Make two subsets of the data, based on the median
    left_split = data[column <= median]
    right_split = data[column > median]
    
    # Loop through the splits and calculate the subset entropies
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0]) 
        to_subtract += prob * calc_entropy(subset[target_name])
    
    # Return information gain
    return original_entropy - to_subtract

columns = ["age", "workclass", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "hours-per-week", "native-country"]

information_gains = []
for col in columns:
    information_gains.append(calc_information_gain(income, col, "high-income"))

index = information_gains.index(max(information_gains))
highest_gain = columns[index]
highest_gain

'marital-status'

In [ ]:
The variable 'marital-status' is therefore the best variable to split the first node on.

We can figure out how to recursively construct an entire tree using the ID3 algorithm.